In [1]:
import pandas as pd
import markovify 
import spacy
import re

import warnings
warnings.filterwarnings('ignore')

from time import time
import gc

In [2]:
data = pd.read_csv("/Users/dsg281/Downloads/grammer_kaggl/GrammarandProductReviews.csv",nrows=1000)

In [3]:
data.shape

(1000, 25)

In [4]:
data.head()

,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,name,...,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,upc
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.025370e+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,Pink Friday: Roman Reloaded Re-Up (w/dvd),...,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,Joshua,6.02537E+11
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,7.341600e+10,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,NaN,NaN,Dorothy W,73416000391
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,7.341600e+10,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,NaN,NaN,Dorothy W,73416000391
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,6.798193e+10,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,113026909.0,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,NaN,NaN,Rebecca,67981934427
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,6.798193e+10,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,171267657.0,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,Walker557,67981934427


In [6]:
data['reviews.text'][:5]

0    i love this album. it's very good. more to the...
1    Good flavor. This review was collected as part...
2                                         Good flavor.
3    I read through the reviews on here before look...
4    My husband bought this gel for us. The gel cau...
Name: reviews.text, dtype: object

In [7]:
Reviews = data['reviews.text']

In [9]:
start_time = time()
comments_generator = markovify.Text(Reviews, state_size = 5)
print("Run time for training the generator : {} seconds".format(round(time()-start_time, 2)))


Run time for training the generator : 0.18 seconds


In [10]:
# Print randomly-generated comments using the built model
def generate_comments(generator, number=10, short=False):
    count = 0
    while count < number:
        if short:
            comment = generator.make_short_sentence(140)
        else:
            comment = generator.make_sentence()
        if comment:
            count += 1
            print("Comment {}".format(count))
            print(comment)
            print()
    

In [11]:
generate_comments(comments_generator)


Comment 1
I have used this product for years and it was terrific.

Comment 2
my sister has been using this product for about a year.

Comment 3
This product goes on smooth and makes your skin feel so new and refreshed.love this cream IS PERFECT.

Comment 4
It smells so good omg, it leaves your skin feeling soft and moisturized all day.

Comment 5
Have the ingredients in Windex window cleaner been changed in the past few years and the scowl line between my eyebrows was nearly gone.

Comment 6
I absolutely LOVE this product!! It does what it says it will do I have used it for many years.

Comment 7
Thanks to Olay!! I love all Olay products that I have used but like this particular one the best.

Comment 8
I have used this product for years and it was terrific.

Comment 9
Of all the moisturizers I've used, Regenerist Deep Hydration Regenerating Cream has been a huge help in staying young looking.

Comment 10
Of all the moisturizers I've used, Regenerist Deep Hydration Regenerating Cream i

Improving Markov chain generator using spaCy for POS-Tagging:¶


In [12]:
nlp = spacy.load("en")

class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

In [14]:
start_time = time()
comments_generator_POSified = POSifiedText(Reviews, state_size = 2)
print("Run time for training the generator : {} seconds".format(round(time()-start_time, 2)))


Run time for training the generator : 24.81 seconds


In [15]:
generate_comments(comments_generator_POSified)


Comment 1
I love this product over the new one but it is a wonderful product because that is my all time favorite product .

Comment 2
This is a problem on a black surface .

Comment 3
The skin does nt feel oily or a heavy lotion .

Comment 4
Used a few out before I found myself returning back to this one is just as good .

Comment 5
Great product ! ! So the Regenerist line .

Comment 6
It goes on great on your lips - it tastes terrible !

Comment 7
I am 81 years old my grandmother told me to minimize aging due to allergies .

Comment 8
I have seen avid results and I have only a week I can do .

Comment 9
Morning and night .

Comment 10
I have found that this is the worst product .

